Con este Notebook vamos a buscar los parámetros ideales para acercarnos a las masas de Neutrón y Delta adecuadas.

In [ ]:
#Importamos las librerias.
import numpy as np
from numba import njit
import matplotlib.pyplot as plt
import matplotlib as mpl
import math
import scipy.integrate as integrate
from tqdm import tqdm #Barra de progreso

#Proporciones y tamaños en las graficas.
params = {'xtick.labelsize': 30, 'ytick.labelsize': 20, 'font.size': 30}#
mpl.rcParams.update(params)

#Parametros iniciales de prueba (en MeV o adimensionales).
F_pi_val = 108 # o 104.8 o 186
e_val = 4.684 # o 4.84
m_pi = 138
alpha = 0.0444
m_rho = 796
kappa_inv = 0.1886 /(np.pi**2/np.sqrt(3))

#Factores de conversion de longitud, energia y energia a longitud.
Factor_l = (2/(e_val *F_pi_val))
Factor_e = F_pi_val/(4*e_val)* (12*np.pi**2)

convers = 806554.815355*10**6 * (2*np.pi) #1MeV = ... m^-1

#Listas para las integrales. lam == lambda^2
n_r = 500 #5000 #Numero de distancias radiales para las integrales.
r_f = 30. #30 #Distancia radial maxima usada (Unidades Skyrme).
n_lam = 100 #Numero de valores  de lambda a consultar.
lam_f = np.sqrt(1.45) + 0.1 #Lambda^2 maximo a consultar (Unidades Skyrme).

#Listas en unidades MeV^-1 y MeV^-2 respectivamente.
rs = np.linspace(1e-6, r_f, n_r)* Factor_l
lams = np.linspace((lam_f-0.1*2), lam_f, n_lam)**2* Factor_l**2

#Tolerancia para los parámetros
Tol_N = 0.49
Tol_D = 0.49
Tol_r = 0.0049

Definimos las funciones que vamos a utilizar.
Ponemos el njit para mejorar MUY notablemente su rendimiento.

Definimos:
* Masa de Skyrme $M_1+M_2$
* Funciones auxiliares para las integrales de las $k_i(r)$
* Integrandos de las $k_i(r)$
* Integrandos de las $M_4$, $M_5$, $M_6$ y $M_7$ y las funciones para integrarlas.

In [ ]:
@njit
def Mskyrme(F_pi_val, lam, e_val):
    return 6.336630541*F_pi_val**2*np.sqrt(lam) + 53.4570253/(e_val**2*np.sqrt(lam))
@njit
def eta(h, r, lam): return h/(h**2+r**2+lam)
@njit
def zeta(h, r, lam): return -r/(h**2+r**2+lam)
@njit
def pm(h): return 0.5 + 0.5*math.erf( h / np.sqrt(2.) )
@njit
def p0(h): return np.exp(-0.5*h*h) / np.sqrt(np.sqrt(np.pi))
@njit
def f(r, lam): return - np.pi / np.sqrt( 1. + lam / (r*r) )
@njit
def F(h, r, lam): return  - 0.5 * r * ( np.pi + 2.*math.atan( h / np.sqrt(r**2+lam) ) )/ (np.sqrt(r**2+lam))
@njit
def k1(h, r, lam): return ( eta(h, r, lam)*np.cos(2*F(h, r, lam)) - (zeta(h, r, lam)+0.5/r)*np.sin(2*F(h, r, lam)) + 0.5*np.sin(2*f(r, lam))*pm(h)/r ) * p0(h)
@njit
def Fdiff(h, r, lam): return  F(h, r, lam) *( 1./r - r/(lam+r*r)) + eta(h, r, lam)*r*r/(lam+r*r)
@njit
def k2(h, r, lam): return ( eta(h, r, lam) -  Fdiff(h, r, lam)   + f(r, lam) * (1 - (f(r, lam) /np.pi )**2 ) / r  * pm(h) )*p0(h)
@njit
def k3(h, r, lam): return ( eta(h, r, lam)*np.sin(2*F(h, r, lam)) + \
            (zeta(h, r, lam)+0.5/r)*np.cos(2*F(h, r, lam)) -\
            0.5/r - 0.5*( np.cos(2*f(r, lam)) - 1 )*pm(h)/r ) *p0(h)
#-----------------------------------------------------------------------------------------------------------
@njit
def M4_dif(k11, k12, k31, k32, dr, r):
    dk1 = (k12-k11)/dr; dk3 = (k32-k31)/dr; k1r = (k12+k11)/2; k3r = (k32+k31)/2#Definimos tanto las derivadas (dk1, dk3) como los valores medios entre los dos puntos(k1r, k3r)
    return (k1r + r*dk1)**2 + 3*k3r**2 + 2*r*k3r*dk3 + dk3**2 * r**2
@njit
def M5_dif(k1, k3, r): return (k1**2 + k3**2)*r**2
@njit
def M6_dif(k31, k32, dr, r, lam):
    dk3 = (k32-k31)/dr; k3r = (k32+k31)/2; a = np.sqrt(lam+r**2)
    return np.sin(np.pi*r/a) / (r*a**3) * ( 2*np.pi*lam*r*(k3r+r*dk3)  + k3r*a**3*np.sin(2*np.pi*r/a) )
@njit
def integrador4(k1, k3, rs):
    dr = rs[1] - rs[0]; M4 = 0
    for i in range(len(rs)-1): M4 += M4_dif(k1[i], k1[i+1], k3[i], k3[i+1], dr, (rs[i]+rs[i+1])/2 ) * dr#Vamos a cada punto y evaluamos el integrando
    return M4*4.*np.pi
@njit
def integrador5(k1, k3, rs):
    dr = rs[2] - rs[1]; M5 = 0
    for i in range(len(rs)): M5 += M5_dif(k1[i], k3[i], rs[i]) * dr#Vamos a cada punto y evaluamos el integrando
    return M5*4.*np.pi
@njit
def integrador6(k3, rs, lam):
    dr = rs[2] - rs[1]; M6 = 0
    for i in range(len(rs)-1): M6 += M6_dif( k3[i], k3[i+1], dr, (rs[i]+rs[i+1])/2, lam ) * dr#Vamos a cada punto y evaluamos el integrando
    return M6*16.*np.pi
@njit
def M7(F_pi_val, lam, kapp_inv): return 4*np.pi**3 * 1.35209 * kapp_inv**2/ (np.sqrt(lam)**3 *F_pi_val**2)
#-----------------------------------------------------------------------------------------------------------

Ahora creamos la función que nos da el mínimo de energía del solitón y su $\lambda$ para unos parámetros dados.

In [ ]:
@njit
def masasgenerator1(F_pi_val, e_val, alpha, kappa_inv, m_rho, rs, lams, n_r, n_lam, k1_r_lam, k3_r_lam):
    #Insertamos estos resultados en los integradores para cada lambda^2.
    M12 = np.array([Mskyrme(F_pi_val, _lam, e_val) for _lam in lams])
    M4 = np.array( [ integrador4(k1_r_lam[_], k3_r_lam[_], rs) for _ in range(n_lam) ] )
    M5 = m_rho**2 * np.array( [ integrador5(k1_r_lam[_], k3_r_lam[_], rs) for _ in range(n_lam) ] )
    M6 = alpha*np.array( [ integrador6(k3_r_lam[_], rs, lams[_]) for _ in range(n_lam) ] )
    M_7 = np.array([M7(F_pi_val, _lam, kappa_inv) for _lam in lams])

    MT = M12 + M4 + M5 + M6 + M_7#Sumamos todas las contribuciones
    
    
    minM = min(MT); indexmin = list(MT).index(minM); minlam = lams[indexmin] #Buscamos tanto la masa minima como su lam correspondiente.

    return minM, minlam, indexmin
#-----------------------------------------------------------------------------------------------------------
@njit
def masasgenerator2(F_pi_val, e_val, kappa_inv, lams, n_lam):
    M12 = np.array([Mskyrme(F_pi_val, _lam, e_val) for _lam in lams])
    M_7 = np.array([M7(F_pi_val, _lam, kappa_inv) for _lam in lams])

    MT = M12 + M_7 #Sumamos todas las contribuciones

    minM = min(MT); minlam = lams[list(MT).index(minM)] #Buscamos tanto la masa minima como su lam correspondiente.

    return minM, minlam

Con esto, ahora crearemos la función para generar los momentos de inercia.

In [ ]:
@njit
def I12(F_pi_val, e_val, lam): return np.pi*np.sqrt(lam)/(e_val**2)*(9.982 + 516.357*F_pi_val**2*e_val**2*lam)
@njit
def I4(k1, k3, rs): return integrador5(k1, k3, rs)*(4./3.)
@njit
def MI6_dif(k3, dr, r, lam):
    a = np.sqrt(lam+r**2)
    return r*r*np.sin(np.pi*r/a)*k3*( np.sin(2.*np.pi*r/a)/r + 2.*np.pi*lam/a**3)
@njit
def integradorI6(k3, rs, lam):
    dr = rs[2] - rs[1]; M6 = 0
    for i in range(len(rs)): M6 += MI6_dif( k3[i], dr, rs[i], lam ) * dr
    return M6
@njit
def I6(alpha, k3, lam, rs):
    return 32*np.pi/3 * alpha * integradorI6(k3, rs, lam)
@njit
def I7(F_pi_val, kappa_inv, lam): return 1.26611*np.pi**3 *kappa_inv**2/(np.sqrt(lam)*F_pi_val**2)

Ahora hacemos funciones para los radios.

In [ ]:
@njit
def rbar(lam): return 0.55718 * lam
@njit
def r_cuad_med_bar(lam, convers): return np.sqrt(rbar(lam))/convers *10**15 #Convertimos a fm

@njit
def rmagn(lam): return 1.809321 * lam 
@njit
def r_cuad_med_magn(lam, convers): return np.sqrt(rmagn(lam))/convers *10**15 #Convertimos a fm

Con estas funciones podemos crear una función que calcule el momento de inercia total para cada conjunto de parámetros.

In [ ]:
@njit
def inerciagenerator1(F_pi_val, e_val, alpha, kappa_inv, rs, lam, n_r, k1_r_lam, k3_r_lam):
    # Introducimos nuestro resultado en las funciones.
    I_12 = I12(F_pi_val, e_val, lam)
    I_4 = I4(k1_r_lam, k3_r_lam, rs)
    I_6 = alpha * I6(alpha, k3_r_lam, lam, rs )
    I_7 = I7(F_pi_val, kappa_inv, lam)

    IT = I_12 + I_4 + I_6 + I_7
    return IT
#-----------------------------------------------------------------------------------------------------------
@njit
def inerciagenerator2(F_pi_val, e_val, kappa_inv, lam):
    I_12 = I12(F_pi_val, e_val, lam)
    I_7 = I7(F_pi_val, kappa_inv, lam)

    IT = I_12 + I_7

    return IT

Ahora construimos una función que nos de la masa del neutrón y la masa del Delta. Para ello calcularemos la masa del solitón y el momento de inercia.

In [ ]:
@njit
def Masasfisicas1(F_pi_val, e_val, alpha, kappa_inv, rs, lams, n_r, n_lam, k1_r_lam, k3_r_lam):
    Factor_e = F_pi_val/(4*e_val)* (12*np.pi**2)

    Masa, lam_min, indexmin = masasgenerator1(F_pi_val, e_val, alpha, kappa_inv, m_rho, rs, lams, n_r, n_lam, k1_r_lam, k3_r_lam)
    Inercia = inerciagenerator1(F_pi_val, e_val, alpha, kappa_inv, rs, lam_min, n_r, k1_r_lam[indexmin], k3_r_lam[indexmin])/Factor_e

    M_N = Masa + 3./(8.*Inercia)
    M_D = Masa + 15./(8.*Inercia)

    return M_N, M_D, lam_min, Masa, Inercia
#-----------------------------------------------------------------------------------------------------------
@njit
def Masasfisicas2(F_pi_val, e_val, kappa_inv, lams, n_lam):
    Factor_e = F_pi_val/(4*e_val)* (12*np.pi**2)
    
    Masa, lam_min = masasgenerator2(F_pi_val, e_val, kappa_inv, lams, n_lam)
    Inercia = inerciagenerator2(F_pi_val, e_val, kappa_inv, lam_min)/Factor_e

    M_N = Masa + 3./(8.*Inercia)
    M_D = Masa + 15./(8.*Inercia)

    return M_N, M_D, lam_min, Masa, Inercia

Ahora vamos a hacer una función que vaya explorando los diferentes valores para las constantes.

Nuestra condición de optimización será que la masa del neutrón, la masa del delta y el radio bariónico sean lo más correctos posible.

In [ ]:
def busca_minimos1(F_pi_val, e_val, alpha, kappa_inv, rs, lams, n_r, n_lam, dF_pi_val, de_val, dalpha, dkappa_inv, nF, ne, na, nk, Tol_N, Tol_D, Tol_r):
    #Nuestras listas de valores para los parametros.
    Fs = np.linspace(F_pi_val-dF_pi_val, F_pi_val+dF_pi_val, nF)
    es = np.linspace(e_val-de_val, e_val+de_val, ne)
    als = np.linspace(alpha-dalpha, alpha+dalpha, na)
    ks = np.linspace(kappa_inv-dkappa_inv, kappa_inv+dkappa_inv, nk)

    convers = 806554.815355*10**6 * (2*np.pi) #1MeV = ... m^-1
    total_iter = nF*ne*na*nk
    #Valores reales.
    M_N_corr = 939 #MeV
    M_D_corr = 1232 #MeV
    r_bar_corr = 0.72 #fm
    #m_magn_corr = 2.79 #fm

    #Nuestos valores guardados (Los inicializamos de manera absurda para poder visualizar problemas).
    M_N_our = 1000000; M_D_our = 1000000; r_bar_our = 1000; r_magn_our = 1000; lam_min_our = 100000; Masa_our = 100000; Inercia_our = 100000
    F_our = 0; e_our = 0; a_our = 0; k_our= 0; m_magn_p_our = 100000; m_magn_n_our = 100000; 

    #Calculamos las ki en cada punto para nuestro lambda^2.
    k1_r_lam = np.array([ [ integrate.quad(lambda h: k1(h, r=_r, lam=_l), -np.inf, np.inf)[0] for _r in rs ] for _l in lams ])
    k3_r_lam = np.array([ [ integrate.quad(lambda h: k3(h, r=_r, lam=_l), -np.inf, np.inf)[0] for _r in rs ] for _l in lams ])

    # Vamos a explorar todas las posibles combinaciones.
    control = 0
    with tqdm(total=100, leave=None, position=0) as pbar:
        for _Fs in Fs:
            for _es in es:
                for _als in als:
                    for _ks in ks:
                        M_N, M_D, lam_min, Masa, Inercia = Masasfisicas1(_Fs, _es, _als, _ks, rs, lams, n_r, n_lam, k1_r_lam, k3_r_lam)
                        r_bar = r_cuad_med_bar(lam_min, convers)
                        r_magn = r_cuad_med_magn(lam_min, convers)
                        #Tambien calculamos los momentos magneticos
                        m_magn_p = M_N/3*((r_bar*convers *10**-15)**2/(2*Inercia) + Inercia)
                        m_magn_n = M_N/3*((r_bar*convers *10**-15)**2/(2*Inercia) - Inercia)
                        # Condicion de mejora en todos los parametros
                        if abs(M_N - M_N_corr) <= abs(M_N_our - M_N_corr)+Tol_N and abs(M_D - M_D_corr) <= abs(M_D_our - M_D_corr)+Tol_D and abs(r_bar - r_bar_corr) <= abs(r_bar_our - r_bar_corr)+Tol_r:# and abs(r_bar - r_bar_corr) <= abs(m_magn_p_our - m_magn_p_corr)+Tol_m:
                            M_N_our = M_N ; M_D_our = M_D ; r_bar_our = r_bar ; r_magn_our = r_magn ; lam_min_our = lam_min ; Masa_our = Masa ; Inercia_our = Inercia
                            F_our = _Fs; e_our = _es; a_our = _als; k_our = _ks; m_magn_p_our = m_magn_p; m_magn_n_our = m_magn_n;
                        control += 1
                        if (control*100) % total_iter == 0: pbar.update(1)

    
    return M_N_our, M_D_our, r_bar_our, r_magn_our, lam_min_our, Masa_our, Inercia_our, F_our, e_our, a_our, k_our, m_magn_p_our, m_magn_n_our
#-----------------------------------------------------------------------------------------------------------
@njit
def busca_minimos2(F_pi_val, e_val, kappa_inv, lams, n_lam, dF_pi_val, de_val, dkappa_inv, nF, ne, nk, Tol_N, Tol_D, Tol_r):
    #Nuestras listas de valores para los parametros.
    Fs = np.linspace(F_pi_val-dF_pi_val, F_pi_val+dF_pi_val, nF)
    es = np.linspace(e_val-de_val, e_val+de_val, ne)
    ks = np.linspace(kappa_inv-dkappa_inv, kappa_inv+dkappa_inv, nk)

    convers = 806554.815355*10**6 * (2*np.pi) #1MeV = ... m^-1
    #Valores reales.
    M_N_corr = 939 #MeV
    M_D_corr = 1232 #MeV
    r_bar_corr = 0.72 #fm

    #Nuestos valores guardados (Los inicializamos de manera absurda para poder visualizar problemas).
    M_N_our = 1000000; M_D_our = 1000000; r_bar_our = 1000; r_magn_our = 1000; lam_min_our = 100000; Masa_our = 100000; Inercia_our = 100000
    F_our = 0; e_our = 0; k_our = 0; m_magn_p_our = 100000; m_magn_n_our = 100000; 
    
    # Vamos a explorar todas las posibles combinaciones.
    for _Fs in Fs:
        for _es in es:
            for _ks in ks:
                M_N, M_D, lam_min, Masa, Inercia = Masasfisicas2(_Fs, _es, _ks, lams, n_lam)
                r_bar = r_cuad_med_bar(lam_min, convers)
                r_magn = r_cuad_med_magn(lam_min, convers)
                #Tambien calculamos los momentos magneticos
                m_magn_p = M_N/3*((r_bar*convers *10**-15)**2/(2*Inercia) + Inercia)
                m_magn_n = M_N/3*((r_bar*convers *10**-15)**2/(2*Inercia) - Inercia)
                # Condicion de mejora en todos los parametros
                if abs(M_N - M_N_corr) <= abs(M_N_our - M_N_corr)+Tol_N and abs(M_D - M_D_corr) <= abs(M_D_our - M_D_corr)+Tol_D and abs(r_bar - r_bar_corr) <= abs(r_bar_our - r_bar_corr)+Tol_r:
                    M_N_our = M_N ; M_D_our = M_D ; r_bar_our = r_bar ; r_magn_our = r_magn ; lam_min_our = lam_min ; Masa_our = Masa ; Inercia_our = Inercia
                    F_our = _Fs; e_our = _es; k_our = _ks; m_magn_p_our = m_magn_p; m_magn_n_our = m_magn_n;


    return M_N_our, M_D_our, r_bar_our, r_magn_our, lam_min_our, Masa_our, Inercia_our, F_our, e_our, k_our, m_magn_p_our, m_magn_n_our

Ahora hacemos unas cuantas pruebas con ciertos valores de los parámetros.

Modelo séxtico.

In [ ]:
dF_pi_val, de_val, dkappa_inv, nF, ne, nk = 30, 10, 0.03, 100, 100, 100

M_N, M_D, r_bar, r_magn, lam_min, Masa, Inercia, F, e, k, m_magn_p, m_magn_n = busca_minimos2(F_pi_val, e_val, kappa_inv, lams, n_lam, dF_pi_val, de_val, dkappa_inv, nF, ne, nk, Tol_N, Tol_D, Tol_r)
print('Masa Neutron ', round(M_N,1))
print('Masa Delta ', round(M_D,1))
print('radio bar = ', round(r_bar,4))
print('radio mag = ', round(r_magn,4))
print()

print('F = ', round(F,3))
print('e = ', round(e,6))
print('kappa = ', round(k,8))
print()

print('Momento magnetico p: ', round(m_magn_p,3))
print('Momento magnetico n: ', round(m_magn_n,3))

print()
print('Masa ', Masa)
print('Inercia ', Inercia)
print('lam_min ', lam_min /Factor_l**2)#Mal reescalado, es para comprobar que no salga de los limites.
print('lam_min ', lam_min /((2/(e *F)))**2)#Bien reescalado

Masa Neutron  938.7
Masa Delta  1231.8
radio bar =  0.7336
radio mag =  1.3219

F =  125.879
e =  5.997131
kappa =  0.02794655

Momento magnetico p:  2.024
Momento magnetico n:  -1.179

Masa  865.4113161814166
Inercia  0.005116896319972036
lam_min  1.5868819150730011
lam_min  3.53391051421065


In [ ]:
dF_pi_val, de_val, dkappa_inv, nF, ne, nk = np.pi, np.pi*1, np.pi*0.01, 100, 100, 100

M_N, M_D, r_bar, r_magn, lam_min, Masa, Inercia, F, e, k, m_magn_p, m_magn_n = busca_minimos2(129.6, 6.37, 0.03, lams, 100, dF_pi_val, de_val, dkappa_inv, nF, ne, nk, Tol_N, Tol_D, Tol_r)
print('Masa Neutron ', round(M_N,1))
print('Masa Delta ', round(M_D,1))
print('radio bar = ', round(r_bar,4))
print('radio mag = ', round(r_magn,4))
print()

print('F = ', F)
print('e = ', e)
print('kappa = ', k)
print()

print('Momento magnetico p: ', round(m_magn_p,3))
print('Momento magnetico n: ', round(m_magn_n,3))

print()
print('Masa ', Masa)
print('Inercia ', Inercia)
print('lam_min ', lam_min /Factor_l**2)#Mal reescalado, es para comprobar que no salga de los limites.
print('lam_min ', lam_min /((2/(e *F)))**2)#Bien reescalado

Masa Neutron  939.2
Masa Delta  1231.5
radio bar =  0.7183
radio mag =  1.2944

F =  128.5528024488034
e =  6.274800222618492
kappa =  0.029682667408728304

Momento magnetico p:  2.011
Momento magnetico n:  -1.203

Masa  866.1714023302004
Inercia  0.005132131142945949
lam_min  1.5214047914206976
lam_min  3.8683556883718864


El bueno.

In [ ]:
dF_pi_val, de_val, dkappa_inv, nF, ne, nk = np.pi*0., np.pi*0., np.pi*0.0, 100, 100, 100

M_N, M_D, r_bar, r_magn, lam_min, Masa, Inercia, F, e, k, m_magn_p, m_magn_n = busca_minimos2(128.55, 6.275, 0.0297, np.linspace((np.sqrt(1.45)-0.1), np.sqrt(1.45) +0.1, 100)**2* Factor_l**2, 100, dF_pi_val, de_val, dkappa_inv, nF, ne, nk, Tol_N, Tol_D, Tol_r)
print('Masa Neutron ', round(M_N,1))
print('Masa Delta ', round(M_D,1))
print('radio bar = ', round(r_bar,4))
print('radio mag = ', round(r_magn,4))
print()

print('F = ', round(F,3))
print('e = ', round(e,6))
print('kappa = ', round(k,8))
print()

print('Momento magnetico p: ', round(m_magn_p,3))
print('Momento magnetico n: ', round(m_magn_n,3))

print()
print('Masa ', Masa)
print('Inercia ', Inercia)
print('lam_min ', lam_min /Factor_l**2)#Mal reescalado, es para comprobar que no salga de los limites.
print('lam_min ', lam_min /((2/(e *F)))**2)#Bien reescalado

Masa Neutron  939.3
Masa Delta  1231.6
radio bar =  0.7183
radio mag =  1.2944

F =  128.55
e =  6.275
kappa =  0.0297

Momento magnetico p:  2.011
Momento magnetico n:  -1.203

Masa  866.2248372227913
Inercia  0.005132183415837736
lam_min  1.5214047914206976
lam_min  3.8684333449984836


No he conseguido un ajuste mejor.

Modelo séxtico con $\rho$. (Si no arranca por algún error, vuelve a correr la celda donde definimos k1, k3, ...)

In [ ]:
dF_pi_val, de_val, dalpha, dkappa_inv, nF, ne, na, nk = 50, 10, 0.04, 0.5, 10, 10, 10, 10

lam_f = 1.5 + 0.5 #Lambda^2 maximo a consultar (Unidades Skyrme).
lams = np.linspace((lam_f-0.5*2), lam_f, n_lam)**2* Factor_l**2

M_N, M_D, r_bar, r_magn, lam_min, Masa, Inercia, F, e, a, k, m_magn_p, m_magn_n = busca_minimos1(F_pi_val, e_val, alpha, kappa_inv, rs, lams, n_r, n_lam, dF_pi_val, de_val, dalpha, dkappa_inv, nF, ne, na, nk, Tol_N, Tol_D, Tol_r)
print()
print('Masa Neutron ', round(M_N,1))
print('Masa Delta ', round(M_D,1))
print('radio bar = ', round(r_bar,4))
print('radio mag = ', round(r_magn,4))
print()

print('F = ', round(F,3))
print('e = ', round(e,6))
print('alpha = ', round(a, 8))
print('kappa = ', round(k,8))
print()

print('Momento magnetico p: ', round(m_magn_p,3))
print('Momento magnetico n: ', round(m_magn_n,3))

print()
print('Masa ', Masa)
print('Inercia ', Inercia)
print('lam_min ', lam_min /Factor_l**2)#Mal reescalado, es para comprobar que no salga de los limites.
print('lam_min ', lam_min /((2/(e *F)))**2)#Bien reescalado

100%|██████████| 100/100 [00:32<00:00,  3.11it/s]


Masa Neutron  934.5
Masa Delta  1269.9
radio bar =  0.7117
radio mag =  1.2826

F =  124.667
e =  5.795111
alpha =  0.0844
kappa =  -0.02245749

Momento magnetico p:  1.847
Momento magnetico n:  -0.941

Masa  850.7201738282299
Inercia  0.004473405033889625
lam_min  1.4938271604938274
lam_min  3.046795440867202


In [ ]:
dF_pi_val, de_val, dalpha, dkappa_inv, nF, ne, na, nk = np.pi*0.1, np.pi*0.0, np.pi*0.0, np.pi*0.001, 50, 1, 1, 50

n_lam = 20
lam_f = 1.25 + 0.1 #Lambda^2 maximo a consultar (Unidades Skyrme).
lams = np.linspace((lam_f-0.1*2), lam_f, n_lam)**2* Factor_l**2

M_N, M_D, r_bar, r_magn, lam_min, Masa, Inercia, F, e, a, k, m_magn_p, m_magn_n = busca_minimos1(128.55, 6.25, 0.0444, 0.0295, rs, lams, n_r, n_lam, dF_pi_val, de_val, dalpha, dkappa_inv, nF, ne, na, nk, Tol_N, Tol_D, Tol_r)
print()
print('Masa Neutron ', round(M_N,1))
print('Masa Delta ', round(M_D,1))
print('radio bar = ', round(r_bar,4))
print('radio mag = ', round(r_magn,4))
print()

print('F = ', F)
print('e = ', e)
print('alpha = ', a)
print('kappa = ', k)
print()

print('Momento magnetico p: ', round(m_magn_p,4))
print('Momento magnetico n: ', round(m_magn_n,4))

print()
print('Masa ', Masa)
print('Inercia ', Inercia)
print('lam_min ', lam_min /Factor_l**2)#Mal reescalado, es para comprobar que no salga de los limites.
print('lam_min ', lam_min /((2/(e *F)))**2)#Bien reescalado

100%|██████████| 100/100 [00:01<00:00, 67.24it/s]


Masa Neutron  939.3
Masa Delta  1232.2
radio bar =  0.7187
radio mag =  1.2952

F =  128.54358858642127
e =  6.25
alpha =  0.0444
kappa =  0.029435885864212454

Momento magnetico p:  2.009
Momento magnetico n:  -1.1978

Masa  866.0827968807088
Inercia  0.005120945008817918
lam_min  1.5232756232686986
lam_min  3.8420063934533717


In [ ]:
dF_pi_val, de_val, dalpha, dkappa_inv, nF, ne, na, nk = np.pi*0.1, np.pi*0.0, np.pi*0.00, np.pi*0.002, 100, 1, 1, 100

n_lam = 20
lam_f = 1.25 + 0.1 #Lambda^2 maximo a consultar (Unidades Skyrme).
lams = np.linspace((lam_f-0.1*2), lam_f, n_lam)**2* Factor_l**2

M_N, M_D, r_bar, r_magn, lam_min, Masa, Inercia, F, e, a, k, m_magn_p, m_magn_n = busca_minimos1(128.55, 6.25, 0.05, 0.0295, rs, lams, n_r, n_lam, dF_pi_val, de_val, dalpha, dkappa_inv, nF, ne, na, nk, Tol_N, Tol_D, Tol_r)
print()
print('Masa Neutron ', round(M_N,1))
print('Masa Delta ', round(M_D,1))
print('radio bar = ', round(r_bar,4))
print('radio mag = ', round(r_magn,4))
print()

print('F = ', F)
print('e = ', e)
print('alpha = ', a)
print('kappa = ', k)
print()

print('Momento magnetico p: ', round(m_magn_p,4))
print('Momento magnetico n: ', round(m_magn_n,4))

print()
print('Masa ', Masa)
print('Inercia ', Inercia)
print('lam_min ', lam_min /Factor_l**2)#Mal reescalado, es para comprobar que no salga de los limites.
print('lam_min ', lam_min /((2/(e *F)))**2)#Bien reescalado

100%|██████████| 100/100 [00:06<00:00, 15.30it/s]


Masa Neutron  939.1
Masa Delta  1232.0
radio bar =  0.7187
radio mag =  1.2952

F =  128.5468266740873
e =  6.25
alpha =  0.05
kappa =  0.029436533481745658

Momento magnetico p:  2.0085
Momento magnetico n:  -1.1976

Masa  865.8544548745424
Inercia  0.005121073444287128
lam_min  1.5232756232686986
lam_min  3.8421999606370356


Podemos comprobar cuan bien ajustado está con los momentos magnéticos.

In [ ]:
M_N = 939 #MeV
M_D = 1232 #MeV
r_bar = 0.72 #fm

M_N*((r_bar*convers*10**-15)**2/9*(M_D-M_N) +1/2 /(M_D-M_N))

2.009378582840425